In [167]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO  # Add this import

In [6]:
SCORE_DIR = "data/scores"

In [8]:
box_scores = os.listdir(SCORE_DIR)

In [14]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [157]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [169]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [171]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs={'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [173]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [175]:
box_score = box_scores[0]
games = []
base_cols = None

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
    
        maxes = total = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
    
        summary = pd.concat([totals, maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
    
        summary = summary[base_cols]
    
        summaries.append(summary)
    
    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis=1)
    
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    
    full_game = pd.concat([game, game_opp], axis=1)
    
    full_game["Season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]  
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"]  = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 8877
200 / 8877
300 / 8877
400 / 8877
500 / 8877
600 / 8877
700 / 8877
800 / 8877
900 / 8877
1000 / 8877
1100 / 8877
1200 / 8877
1300 / 8877
1400 / 8877
1500 / 8877
1600 / 8877
1700 / 8877
1800 / 8877
1900 / 8877
2000 / 8877
2100 / 8877
2200 / 8877
2300 / 8877
2400 / 8877
2500 / 8877
2600 / 8877
2700 / 8877
2800 / 8877
2900 / 8877
3000 / 8877
3100 / 8877
3200 / 8877
3300 / 8877
3400 / 8877
3500 / 8877
3600 / 8877
3700 / 8877
3800 / 8877
3900 / 8877
4000 / 8877
4100 / 8877
4200 / 8877
4300 / 8877
4400 / 8877
4500 / 8877
4600 / 8877
4700 / 8877
4800 / 8877
4900 / 8877
5000 / 8877
5100 / 8877
5200 / 8877
5300 / 8877
5400 / 8877
5500 / 8877
5600 / 8877
5700 / 8877
5800 / 8877
5900 / 8877
6000 / 8877
6100 / 8877
6200 / 8877
6300 / 8877
6400 / 8877
6500 / 8877
6600 / 8877
6700 / 8877
6800 / 8877
6900 / 8877
7000 / 8877
7100 / 8877
7200 / 8877
7300 / 8877
7400 / 8877
7500 / 8877
7600 / 8877
7700 / 8877
7800 / 8877
7900 / 8877
8000 / 8877
8100 / 8877
8200 / 8877
8300 / 8877
8400 / 8877
8

In [177]:
games_df = pd.concat(games, ignore_index=True)

In [179]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,Season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17749,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17750,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17751,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17752,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [ ]:
games_df.to_csv("nba_games.csv")